<a href="https://colab.research.google.com/github/rmlessa/projeto_final/blob/main/projeto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perguntas de Negócio

1. A renda média do bairro que recebe o abastecimento está diretamente ligada a qualidade da água fornecida?

2. A densidade populacional por bairro está relacionada com a qualidade da água fornecida?

3. Quais bairros de Manaus necessitam de tratamento de água adicional?


**Missão de hoje: **

1. Limpar os dados de 2018 a 2020 e juntar os datasets separados por ano

2. Criar uma coluna no dataset para representar coliformes fecais como ausente e presente sendo que ausente será igual a 0 e presente será igual a 1

3. Verificar a extratificação da porcentagem de coliformes presentes por bairro

In [1]:
#instala o pyspark
!pip install pyspark

     |████████████████████████████████| 212.3MB 65kB/s 
     |████████████████████████████████| 204kB 19.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=91249baab6fe65d9a31358d88f6919a2df32405fc0235302ddcf53db97b84e71
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
#importa bibliotecas que serão utilizadas
import pandas as pd
import sqlite3
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import monotonically_increasing_id #incrementa indice dos dataframes pyspark
from pyspark.sql.functions import lit
from pyspark.sql.functions import trim
from functools import reduce
from pyspark.sql import DataFrame

In [3]:
#monta o google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#cria sessão spark no colab para realizar a extração dos dados dos arquivos countries.csv e flags.csv
if __name__ == '__main__':
    scSpark = SparkSession \
        .builder \
        .appName("etl") \
        .getOrCreate()

In [5]:
#salva o caminho onde o arquivo csv para o ano de 2018 está
analise_agua_2018 = '/content/drive/MyDrive/01 - AQUISIÇÃO DE DADOS/Projeto_Final/2amostras_de_vigilancia_parametros_basicos_anual_2018.csv'

In [6]:
#salva o caminho onde o arquivo csv para o ano de 2019 está
analise_agua_2019 = '/content/drive/MyDrive/01 - AQUISIÇÃO DE DADOS/Projeto_Final/2amostras_de_vigilancia_parametros_basicos_anual_2019.csv'

In [7]:
#salva o caminho onde o arquivo csv para o ano de 2020 está
analise_agua_2020 = '/content/drive/MyDrive/01 - AQUISIÇÃO DE DADOS/Projeto_Final/2amostras_de_vigilancia_parametros_basicos_anual_2020.csv'

In [8]:
#salva o caminho onde o arquivo csv dos dados socioeconômicos estão
dados_soc_eco =  '/content/drive/MyDrive/01 - AQUISIÇÃO DE DADOS/Projeto_Final/DADOS_SOCIOECONOMICOS_BAIRROS_MANAUS.CSV'

In [9]:
#dataframe pyspark para receber dados dos arquivos .csv

#dataset com dados de análises de águas feitas nos municípios brasileiros 2018
session_2018 = scSpark.read.csv(analise_agua_2018, header=True, encoding="ISO-8859-1" ,sep=";").cache() #armazena o resultado em cache para melhor desempenho
print('Total de Registros em 2018.csv = {}'.format(session_2018.count()))
session_2018.show()

Total de Registros em 2018.csv = 2677498
+-----------------+--------+--------------------+--------------------+--------------------+---------------+--------------------+------------------------+--------------------------+------------------------+--------------------+-----------------+--------------------+--------------------+------------------------------+--------------------+--------------------+-----+---+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|REGIAO_GEOGRAFICA|SIGLA_UF|   REGIONAL_DE_SAUDE|      NOME_MUNICIPIO|         CODIGO_IBGE| NUMERO_AMOSTRA|       MOTIVO_COLETA|TIPO_FORMA_ABASTECIMENTO|CODIGO_FORMA_ABASTECIMENTO|NOME_FORMA_ABASTECIMENTO| Tipo da Instituição|Sigla Instituição|

In [10]:
#dataframe pyspark para receber dados dos arquivos .csv

#dataset com dados de análises de águas feitas nos municípios brasileiros 2019
session_2019 = scSpark.read.csv(analise_agua_2019, header=True, encoding="ISO-8859-1" ,sep=";").cache() #armazena o resultado em cache para melhor desempenho
print('Total de Registros em 2019.csv = {}'.format(session_2019.count()))
session_2019.show()

Total de Registros em 2019.csv = 2836968
+-----------------+--------+--------------------+--------------------+--------------------+---------------+--------------------+------------------------+--------------------------+------------------------+--------------------+-----------------+--------------------+--------------------+------------------------------+--------------------+--------------------+-----+---+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|REGIAO_GEOGRAFICA|SIGLA_UF|   REGIONAL_DE_SAUDE|      NOME_MUNICIPIO|         CODIGO_IBGE| NUMERO_AMOSTRA|       MOTIVO_COLETA|TIPO_FORMA_ABASTECIMENTO|CODIGO_FORMA_ABASTECIMENTO|NOME_FORMA_ABASTECIMENTO| Tipo da Instituição|Sigla Instituição|

In [11]:
#dataframe pyspark para receber dados dos arquivos .csv

#dataset com dados de análises de águas feitas nos municípios brasileiros 2020
session_2020 = scSpark.read.csv(analise_agua_2020, header=True, encoding="ISO-8859-1" ,sep=";").cache() #armazena o resultado em cache para melhor desempenho
print('Total de Registros em 2020.csv = {}'.format(session_2020.count()))
session_2020.show()

Total de Registros em 2020.csv = 2125798
+-----------------+--------+--------------------+--------------------+--------------------+---------------+--------------------+------------------------+--------------------------+------------------------+--------------------+-----------------+--------------------+--------------------+------------------------------+--------------------+--------------------+-----+---+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|REGIAO_GEOGRAFICA|SIGLA_UF|   REGIONAL_DE_SAUDE|      NOME_MUNICIPIO|         CODIGO_IBGE| NUMERO_AMOSTRA|       MOTIVO_COLETA|TIPO_FORMA_ABASTECIMENTO|CODIGO_FORMA_ABASTECIMENTO|NOME_FORMA_ABASTECIMENTO| Tipo da Instituição|Sigla Instituição|

In [12]:
#dataframe pyspark para receber dados dos arquivos .csv

#dados socioeconômicos dos bairros de Manaus
dados_bairros_manaus = scSpark.read.csv(dados_soc_eco, header=True, encoding="ISO-8859-1" ,sep=";").cache() #armazena o resultado em cache para melhor desempenho
print('Total de Registros.csv = {}'.format(dados_bairros_manaus.count()))
dados_bairros_manaus.show()

Total de Registros.csv = 426
+------------+--------------------+-------------------+-----------------------+-----------------+-------------+---------------------+
|    ZONA_ADM|              BAIRRO|POPULACAO_IBGE_2010|POPULACAO_ESTIMADA_2020|RENDA_MENSAL_2010|QTD_ELEITORES|PERIMETRO_PMM_2010_HA|
+------------+--------------------+-------------------+-----------------------+-----------------+-------------+---------------------+
|CENTRO-OESTE|            ALVORADA|             64.621|                 79.595|            1.171|       49.752|               553,18|
|CENTRO-OESTE|       BAIRRO DA PAZ|             15.193|                 18.714|            1.464|       11.426|               240,97|
|CENTRO-OESTE|           DOM PEDRO|             17.070|                 21.025|            2.456|       21.217|               275,78|
|CENTRO-OESTE|            PLANALTO|             16.283|                 20.056|            2.500|        7.373|               429,22|
|CENTRO-OESTE|            REDENÇÃ

In [44]:
#remover espaço em branco do valor das colunas

print("Removendo espaços em branco à esquerda e a direita das colunas REGIÃO_GEOGRAFICA, NOME_MUNICIPIO e AREA")

#2018
session_2018 = session_2018.withColumn("REGIAO_GEOGRAFICA", trim(col="REGIAO_GEOGRAFICA"))
session_2018 = session_2018.withColumn("NOME_MUNICIPIO", trim(col="NOME_MUNICIPIO"))
session_2018 = session_2018.withColumn("CATEGORIA_AREA", trim("CATEGORIA_AREA"))
session_2018 = session_2018.withColumn("AREA", trim(col="AREA"))
session_2018 = session_2018.withColumn("ZONA", trim(col="ZONA"))
session_2018 = session_2018.withColumn("RESULTADO", trim(col="RESULTADO"))

#2019
session_2019 = session_2019.withColumn("REGIAO_GEOGRAFICA", trim(col="REGIAO_GEOGRAFICA"))
session_2019 = session_2019.withColumn("NOME_MUNICIPIO", trim(col="NOME_MUNICIPIO"))
session_2019 = session_2019.withColumn("CATEGORIA_AREA", trim("CATEGORIA_AREA"))
session_2019 = session_2019.withColumn("AREA", trim(col="AREA"))
session_2019 = session_2019.withColumn("ZONA", trim(col="ZONA"))
session_2018 = session_2018.withColumn("RESULTADO", trim(col="RESULTADO"))

#2020
session_2020 = session_2020.withColumn("REGIAO_GEOGRAFICA", trim(col="REGIAO_GEOGRAFICA"))
session_2020 = session_2020.withColumn("NOME_MUNICIPIO", trim(col="NOME_MUNICIPIO"))
session_2020 = session_2020.withColumn("CATEGORIA_AREA", trim("CATEGORIA_AREA"))
session_2020 = session_2020.withColumn("AREA", trim(col="AREA"))
session_2020 = session_2020.withColumn("ZONA", trim(col="ZONA"))
session_2018 = session_2018.withColumn("RESULTADO", trim(col="RESULTADO"))

print("Espaços em branco removidos")


Removendo espaços em branco à esquerda e a direita das colunas REGIÃO_GEOGRAFICA, NOME_MUNICIPIO e AREA
Espaços em branco removidos


In [ ]:
#transforma em tabelas para recuperar os dados da região norte, principalmente Manaus
session_2018.createOrReplaceTempView("vw_dados_regiao_norte_2018")

In [52]:
session_2018.show()

+-----------------+--------+--------------------+--------------------+--------------------+---------------+--------------------+------------------------+--------------------------+------------------------+--------------------+-----------------+--------------------+--------------------+------------------------------+--------------------+--------------------+-----+---+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+
|REGIAO_GEOGRAFICA|SIGLA_UF|   REGIONAL_DE_SAUDE|      NOME_MUNICIPIO|         CODIGO_IBGE| NUMERO_AMOSTRA|       MOTIVO_COLETA|TIPO_FORMA_ABASTECIMENTO|CODIGO_FORMA_ABASTECIMENTO|NOME_FORMA_ABASTECIMENTO| Tipo da Instituição|Sigla Instituição| Nome da Instituição|    CNPJ_INSTITUICAO|Nome escrit

In [66]:
#reduz os dados da tabela para trabalharmos apenas com Manaus
dados_manaus_2018 =  scSpark.sql(
    "SELECT ANO, MES, TRIM(AREA) AS AREA, TRIM(NUMERO_AMOSTRA) AS NUMERO_AMOSTRA, TRIM(PARAMETRO) AS PARAMETRO, TRIM(RESULTADO) AS RESULTADO, (CASE WHEN RESULTADO = 'AUSENTE' THEN 0 ELSE 1 END) AS RESULTADO_FLAG  from vw_dados_regiao_norte_2018 where regiao_geografica = 'NORTE' and SIGLA_UF = 'AM' AND NOME_MUNICIPIO = 'MANAUS' and CATEGORIA_AREA = 'Bairro' and (PARAMETRO like '%Coli%' or PARAMETRO like '%coli%') AND ZONA = 'Urbana'  ORDER BY AREA, MES")
print('Total de dados_regiao_norte_2018 = {}'.format(dados_manaus_2018.count()))
dados_manaus_2018.show()

Total de dados_regiao_norte_2018 = 4892
+-----+---+------------+--------------+-----------------+---------+--------------+
|  ANO|MES|        AREA|NUMERO_AMOSTRA|        PARAMETRO|RESULTADO|RESULTADO_FLAG|
+-----+---+------------+--------------+-----------------+---------+--------------+
| 2018| 04|ADRIANOPOLIS|  180001000474| Escherichia coli|  AUSENTE|             0|
| 2018| 04|ADRIANOPOLIS|  180001000469| Escherichia coli|  AUSENTE|             0|
| 2018| 04|ADRIANOPOLIS|  180001000726|Coliformes totais|  AUSENTE|             0|
| 2018| 04|ADRIANOPOLIS|  180001000472|Coliformes totais|  AUSENTE|             0|
| 2018| 04|ADRIANOPOLIS|  180001000731| Escherichia coli|  AUSENTE|             0|
| 2018| 04|ADRIANOPOLIS|  180001000473|Coliformes totais| PRESENTE|             1|
| 2018| 04|ADRIANOPOLIS|  180001000813| Escherichia coli|  AUSENTE|             0|
| 2018| 04|ADRIANOPOLIS|  180001000471|Coliformes totais|  AUSENTE|             0|
| 2018| 04|ADRIANOPOLIS|  180001000729| Escheri

In [67]:
#transforma em tabelas para recuperar os dados da região norte, principalmente Manaus
session_2019.createOrReplaceTempView("vw_dados_regiao_norte_2019")

#reduz os dados da tabela para trabalharmos apenas com Manaus
dados_manaus_2019 =  scSpark.sql(
    "SELECT ANO, MES, TRIM(AREA) AS AREA, TRIM(NUMERO_AMOSTRA) AS NUMERO_AMOSTRA, TRIM(PARAMETRO) AS PARAMETRO, TRIM(RESULTADO) AS RESULTADO, (CASE WHEN RESULTADO = 'AUSENTE' THEN 0 ELSE 1 END) AS RESULTADO_FLAG from vw_dados_regiao_norte_2019 where regiao_geografica = 'NORTE' and SIGLA_UF = 'AM' AND NOME_MUNICIPIO = 'MANAUS' and CATEGORIA_AREA = 'Bairro' and (PARAMETRO like '%Coli%' or PARAMETRO like '%coli%') AND ZONA = 'Urbana' ORDER BY AREA,MES")
print('Total de dados_regiao_norte_2019 = {}'.format(dados_manaus_2019.count()))
dados_manaus_2019.show()


Total de dados_regiao_norte_2019 = 9784
+-----+---+------------+--------------+-----------------+---------+--------------+
|  ANO|MES|        AREA|NUMERO_AMOSTRA|        PARAMETRO|RESULTADO|RESULTADO_FLAG|
+-----+---+------------+--------------+-----------------+---------+--------------+
| 2019| 01|ADRIANOPOLIS|  190001000006| Escherichia coli|  AUSENTE|             1|
| 2019| 01|ADRIANOPOLIS|  190001000004|Coliformes totais|  AUSENTE|             1|
| 2019| 01|ADRIANOPOLIS|  190001000011|Coliformes totais|  AUSENTE|             1|
| 2019| 01|ADRIANOPOLIS|  190001000009| Escherichia coli|  AUSENTE|             1|
| 2019| 01|ADRIANOPOLIS|  190001000004| Escherichia coli|  AUSENTE|             1|
| 2019| 01|ADRIANOPOLIS|  190001000001| Escherichia coli|  AUSENTE|             1|
| 2019| 01|ADRIANOPOLIS|  190001000007| Escherichia coli|  AUSENTE|             1|
| 2019| 01|ADRIANOPOLIS|  190001000001|Coliformes totais|  AUSENTE|             1|
| 2019| 01|ADRIANOPOLIS|  190001000011| Escheri

In [68]:
#transforma em tabelas para recuperar os dados da região norte, principalmente Manaus
session_2020.createOrReplaceTempView("vw_dados_regiao_norte_2020")

#reduz os dados da tabela para trabalharmos apenas com Manaus
dados_manaus_2020 =  scSpark.sql(
    "SELECT ANO, MES, TRIM(AREA) AS AREA, TRIM(NUMERO_AMOSTRA) AS NUMERO_AMOSTRA, TRIM(PARAMETRO) AS PARAMETRO, TRIM(RESULTADO) AS RESULTADO, (CASE WHEN RESULTADO = 'AUSENTE' THEN 0 ELSE 1 END) AS RESULTADO_FLAG   from vw_dados_regiao_norte_2020 where regiao_geografica = 'NORTE' and SIGLA_UF = 'AM' AND NOME_MUNICIPIO = 'MANAUS' and CATEGORIA_AREA = 'Bairro' and (PARAMETRO like '%Coli%' or PARAMETRO like '%coli%') AND ZONA = 'Urbana' ORDER BY AREA,MES, NUMERO_AMOSTRA")
print('Total de dados_regiao_norte_2020 = {}'.format(dados_manaus_2020.count()))
dados_manaus_2020.show()

Total de dados_regiao_norte_2020 = 5671
+-----+---+------------+--------------+-----------------+---------+--------------+
|  ANO|MES|        AREA|NUMERO_AMOSTRA|        PARAMETRO|RESULTADO|RESULTADO_FLAG|
+-----+---+------------+--------------+-----------------+---------+--------------+
| 2020| 01|ADRIANOPOLIS|          0157| Escherichia coli|  AUSENTE|             1|
| 2020| 01|ADRIANOPOLIS|          0157|Coliformes totais|  AUSENTE|             1|
| 2020| 01|ADRIANOPOLIS|  200001000005|Coliformes totais|  AUSENTE|             1|
| 2020| 01|ADRIANOPOLIS|  200001000005| Escherichia coli|  AUSENTE|             1|
| 2020| 01|ADRIANOPOLIS|  200001000006| Escherichia coli|  AUSENTE|             1|
| 2020| 01|ADRIANOPOLIS|  200001000006|Coliformes totais|  AUSENTE|             1|
| 2020| 01|ADRIANOPOLIS|  200001000007|Coliformes totais|  AUSENTE|             1|
| 2020| 01|ADRIANOPOLIS|  200001000007| Escherichia coli|  AUSENTE|             1|
| 2020| 01|ADRIANOPOLIS|  200001000008| Escheri

In [27]:
#após a redução para dados de Manaus, analisei que colunas seriam necessárias para trabalharmos
#comecei identificando que colunas estão COM DADOS NULOS. Identifiquei latitude, longitude e nome_eta


col_latitude = scSpark.sql("SELECT latitude from vw_dados_regiao_norte_2018 where TRIM(REGIAO_GEOGRAFICA) = 'NORTE' and latitude is not null and trim(nome_municipio) = 'MANAUS' ")
print("2018")
col_latitude.show()

col_latitude = scSpark.sql("SELECT latitude from vw_dados_regiao_norte_2019 where TRIM(REGIAO_GEOGRAFICA) = 'NORTE' and latitude is not null and trim(nome_municipio) = 'MANAUS' ")
print("2019")
col_latitude.show()


col_latitude = scSpark.sql("SELECT latitude from vw_dados_regiao_norte_2020 where TRIM(REGIAO_GEOGRAFICA) = 'NORTE' and latitude is not null and trim(nome_municipio) = 'MANAUS' ")
print("2020")
col_latitude.show()


col_longitude = scSpark.sql("SELECT longitude from vw_dados_regiao_norte_2018 where TRIM(REGIAO_GEOGRAFICA) = 'NORTE' and longitude is not null and trim(nome_municipio) = 'MANAUS'")
print("2018")
col_longitude.show()

col_longitude = scSpark.sql("SELECT longitude from vw_dados_regiao_norte_2019 where TRIM(REGIAO_GEOGRAFICA) = 'NORTE' and longitude is not null and trim(nome_municipio) = 'MANAUS'")
print("2019")
col_longitude.show()

col_longitude = scSpark.sql("SELECT longitude from vw_dados_regiao_norte_2020 where TRIM(REGIAO_GEOGRAFICA) = 'NORTE' and longitude is not null and trim(nome_municipio) = 'MANAUS'")
print("2020")
col_longitude.show()

col_nome_eta = scSpark.sql(
    "select distinct nome_eta from vw_dados_regiao_norte_2018 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2018")
col_nome_eta.show()

col_nome_eta = scSpark.sql(
    "select distinct nome_eta from vw_dados_regiao_norte_2019 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2019")
col_nome_eta.show()

col_nome_eta = scSpark.sql(
    "select distinct nome_eta from vw_dados_regiao_norte_2020 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2020")
col_nome_eta.show()


#Acredito que essas colunas podem ser removidas logo de cara (latitude, longitude e nome_eta), 
#uma vez que não possuem dados. Outras colunas que acho que podem ser removida são as colunas:

#REGIAO_GEOGRAFICA: pois só estamos trabalhando apenas com a região norte
#SIGLA_UF.........: pois como estamos trabalhando apenas com Manaus essa coluna não se faz necessária para caracterizar algo
#NOME_MUNICIPIO...: Vai ser sempre MANAUS, então não precisamos dessa coluna
#CODIGO_IBGE......: Esse código é utilizado pelo IBGE como ID do município, ele deve se repetir.
#CNPJ_INSTITUICAO.: não irá influenciar na análise
#CNPJ_ESCRITORIO..: não irá influenciar na análise
#TIPO_LOCAL.......: não irá influenciar na análise e possui muitos valores nulos
#LOCAL............: não irá influenciar na análise e possui muitos valores nulos

#Vou verificar as demais colunas e identificar que colunas possuem um único valor e vou retirá-las

#REGIONAL_SAUDE [REMOVER]
col_regional_saude = scSpark.sql(
    "select distinct regional_de_saude from vw_dados_regiao_norte_2018 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2018")
col_regional_saude.show()

col_regional_saude = scSpark.sql(
    "select distinct regional_de_saude from vw_dados_regiao_norte_2019 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2019")
col_regional_saude.show()

col_regional_saude = scSpark.sql(
    "select distinct regional_de_saude from vw_dados_regiao_norte_2020 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2020")
col_regional_saude.show()



#CODIGO_IBGE [REMOVER]
#=====> RESULTADO DO DISTINCT PARA CODIGO_IBGE = 130260 <=====
col_codigo_ibge = scSpark.sql(
    "select distinct codigo_ibge from vw_dados_regiao_norte_2018 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2018")
col_codigo_ibge.show()

col_codigo_ibge = scSpark.sql(
    "select distinct codigo_ibge from vw_dados_regiao_norte_2019 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2019")
col_codigo_ibge.show()


col_codigo_ibge = scSpark.sql(
    "select distinct codigo_ibge from vw_dados_regiao_norte_2020 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2020")
col_codigo_ibge.show()


#MOTIVO_COLETA [REMOVER]
#=====> RESULTADO DO DISTINCT PARA MOTIVO_COLETA = ROTINA <=====
col_motivo_coleta = scSpark.sql(
    "select distinct motivo_coleta from vw_dados_regiao_norte_2018 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2018")
col_motivo_coleta.show()

col_motivo_coleta = scSpark.sql(
    "select distinct motivo_coleta from vw_dados_regiao_norte_2019 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2019")
col_motivo_coleta.show()

col_motivo_coleta = scSpark.sql(
    "select distinct motivo_coleta from vw_dados_regiao_norte_2020 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2020")
col_motivo_coleta.show()


#ZONA [NÃO REMOVER]
#=====> RESULTADO DO DISTINCT PARA ZONA = URBANA  <=====
col_zona = scSpark.sql(
    "select distinct zona from vw_dados_regiao_norte_2018 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2018")
col_zona.show()

col_zona = scSpark.sql(
    "select distinct zona from vw_dados_regiao_norte_2019 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2019")
col_zona.show()

col_zona = scSpark.sql(
    "select distinct zona from vw_dados_regiao_norte_2020 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2020")
col_zona.show()


#CATEGORIA_AREA [NÃO REMOVER]
#=====> RESULTADO DO DISTINCT PARA CATEGORIA_AREA = BAIRRO <=====
col_categoria_area = scSpark.sql(
    "select distinct categoria_area from vw_dados_regiao_norte_2018 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2018")
col_categoria_area.show()

analise_categoria_area = scSpark.sql(
    "select * from vw_dados_regiao_norte_2018 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS' and trim(categoria_area) = 'Povoado/Lugarejo'"
)
print("2018")
analise_categoria_area.show()

col_categoria_area = scSpark.sql(
    "select distinct categoria_area from vw_dados_regiao_norte_2019 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2019")
col_categoria_area.show()

col_categoria_area = scSpark.sql(
    "select distinct categoria_area from vw_dados_regiao_norte_2020 where trim(regiao_geografica) = 'NORTE' and trim(nome_municipio) = 'MANAUS'"
) 
print("2020")
col_categoria_area.show()



2018
+--------+
|latitude|
+--------+
+--------+

2019
+--------+
|latitude|
+--------+
+--------+

2020
+--------+
|latitude|
+--------+
+--------+

2018
+---------+
|longitude|
+---------+
+---------+

2019
+---------+
|longitude|
+---------+
+---------+

2020
+---------+
|longitude|
+---------+
+---------+

2018
+--------------------+
|            nome_eta|
+--------------------+
|                 ...|
+--------------------+

2019
+--------------------+
|            nome_eta|
+--------------------+
|                 ...|
+--------------------+

2020
+--------------------+
|            nome_eta|
+--------------------+
|                 ...|
+--------------------+

2018
+--------------------+
|   regional_de_saude|
+--------------------+
|UNICA            ...|
+--------------------+

2019
+--------------------+
|   regional_de_saude|
+--------------------+
|UNICA            ...|
+--------------------+

2020
+--------------------+
|   regional_de_saude|
+--------------------+
|UNICA   

In [39]:
#removendo colunas desnecessárias
#tem umas colunas que não removi pq não sei remover colunas tipo: Tipo da Instituição. 
#Não é comum ter esse tipo de nome separados por espaços. 
#Vou transformar em dataframe pandas pra ver se é mais fácil remover

dados_manaus_2018 = reduce(DataFrame.drop,['LATITUDE', 'LONGITUDE', 'NOME_ETA','REGIAO_GEOGRAFICA', 'SIGLA_UF', 'NOME_MUNICIPIO', 'CODIGO_IBGE', 'CNPJ_INSTITUICAO', 'CNPJ_ESCRITORIO', 'TIPO_LOCAL', 'LOCAL', 'REGIONAL_DE_SAUDE', 'MOTIVO_COLETA'],dados_manaus_2018)
dados_manaus_2019 = reduce(DataFrame.drop,['LATITUDE', 'LONGITUDE', 'NOME_ETA','REGIAO_GEOGRAFICA', 'SIGLA_UF', 'NOME_MUNICIPIO', 'CODIGO_IBGE', 'CNPJ_INSTITUICAO', 'CNPJ_ESCRITORIO', 'TIPO_LOCAL', 'LOCAL', 'REGIONAL_DE_SAUDE', 'MOTIVO_COLETA'],dados_manaus_2019)
dados_manaus_2020 = reduce(DataFrame.drop,['LATITUDE', 'LONGITUDE', 'NOME_ETA','REGIAO_GEOGRAFICA', 'SIGLA_UF', 'NOME_MUNICIPIO', 'CODIGO_IBGE', 'CNPJ_INSTITUICAO', 'CNPJ_ESCRITORIO', 'TIPO_LOCAL', 'LOCAL', 'REGIONAL_DE_SAUDE', 'MOTIVO_COLETA'],dados_manaus_2020)

In [70]:
#Transformação dos dataframes em views para juntar os dados
dados_manaus_2018.createOrReplaceTempView("vw_dados_manaus_2018")
dados_manaus_2019.createOrReplaceTempView("vw_dados_manaus_2019")
dados_manaus_2020.createOrReplaceTempView("vw_dados_manaus_2020")

dados_manaus_consolidado = scSpark.sql("select A.ANO, A.MES, A.AREA, A.NUMERO_AMOSTRA, A.PARAMETRO, A.RESULTADO, (CASE WHEN A.RESULTADO = 'AUSENTE' THEN 0 ELSE 1 END) AS RESULTADO_FLAG from vw_dados_manaus_2018 A union select B.ANO, B.MES, B.AREA, B.NUMERO_AMOSTRA, B.PARAMETRO, B.RESULTADO, (CASE WHEN B.RESULTADO = 'AUSENTE' THEN 0 ELSE 1 END) AS RESULTADO_FLAG from vw_dados_manaus_2019 B union select C.ANO, C.MES, C.AREA, C.NUMERO_AMOSTRA, C.PARAMETRO, C.RESULTADO, (CASE WHEN C.RESULTADO = 'AUSENTE' THEN 0 ELSE 1 END) AS RESULTADO_FLAG from vw_dados_manaus_2020 C")
print('Total de dados_manaus_consolidade = {}'.format(dados_manaus_consolidado.count()))
dados_manaus_consolidado.show()

Total de dados_manaus_consolidade = 20347
+-----+---+-------------------+--------------+-----------------+---------+--------------+
|  ANO|MES|               AREA|NUMERO_AMOSTRA|        PARAMETRO|RESULTADO|RESULTADO_FLAG|
+-----+---+-------------------+--------------+-----------------+---------+--------------+
| 2018| 10|           ALVORADA|  180001001412| Escherichia coli|  AUSENTE|             0|
| 2018| 05|          APARECIDA|  180001000840| Escherichia coli|  AUSENTE|             0|
| 2018| 11|     ARMANDO MENDES|  180001001886|Coliformes totais|  AUSENTE|             0|
| 2018| 04|       CACHOEIRINHA|  180001000702|Coliformes totais|  AUSENTE|             0|
| 2018| 04|             CENTRO|  180001000541|Coliformes totais|  AUSENTE|             0|
| 2018| 04|             CENTRO|  180001000611|Coliformes totais|  AUSENTE|             0|
| 2018| 04|        CIDADE NOVA|          0671|Coliformes totais| PRESENTE|             1|
| 2018| 10|        CIDADE NOVA|  180001001861| Escherichia

In [71]:
#cria a view com dados de 2018 a 2020 para que possa ser realizada a análise
dados_manaus_consolidado.createOrReplaceTempView("vw_manaus_consolidado")

In [ ]:
analise_amostras_bairro = "select ano, bairro, count(*) qtd_"

In [19]:
#transforma em dataframe padas
df_dados_manaus_2020 = dados_manaus_2020.toPandas()

In [20]:
#renomeia as colunas que estavam com nome separado por espaços em branco
#axis = 1 indica que queremos altera o label da coluna. Se fosse 0 alteraríamos a label do índice
df_dados_manaus_2020 = df_dados_manaus_2020.rename(
    {'Tipo da Instituição': 'TIPO_INSTITUICAO' , 
     'Sigla Instituição'  : 'SIGLA_INSTITUICAO', 
     'Nome da Instituição': 'NOME_INSTITUICAO' ,
     'Nome escritório regional/local': 'NOME_ESCRITORIO_REGIONAL',
     "'Análise realizada'": 'ANALISE_REALIZADA',
     "'Providência'"  : 'PROVIDENCIA'}, axis = 1) 

In [21]:
df_dados_manaus_2020.tail()

,NUMERO_AMOSTRA,TIPO_FORMA_ABASTECIMENTO,CODIGO_FORMA_ABASTECIMENTO,NOME_FORMA_ABASTECIMENTO,TIPO_INSTITUICAO,SIGLA_INSTITUICAO,NOME_INSTITUICAO,NOME_ESCRITORIO_REGIONAL,ANO,MES,DATA_COLETA,DATA_LAUDO,DATA_REGISTRO_SISAGUA,PROCEDENCIA_COLETA,PONTO_COLETA,DESCRICAO_LOCAL,AREA,PARAMETRO,ANALISE_REALIZADA,RESULTADO,PROVIDENCIA
16272,2353,SAA,S130260000007,SISTEMA DE ABASTECIMENTO MANAUS ...,Serviço Municipal e outros ...,,MANAUS AMBIENTAL SA ...,...,2020,10,2020/10/20 00:00:00.000,2020/10/21 00:00:00.000,2020/11/29 18:25:17.000,SISTEMA DE DISTRIBUIÇÃO ...,Torneira antes da reservação ...,"RUA BOM JESUS, N 259 ...",ZUMBI DOS PALMARES,Escherichia coli ...,...,AUSENTE ...,...
16273,2349,SAA,S130260000007,SISTEMA DE ABASTECIMENTO MANAUS ...,Serviço Municipal e outros ...,,MANAUS AMBIENTAL SA ...,...,2020,10,2020/10/20 00:00:00.000,2020/10/21 00:00:00.000,2020/11/29 18:04:28.000,SISTEMA DE DISTRIBUIÇÃO ...,Torneira antes da reservação ...,"RUA ANTENOR CAVALCANTE, N 59 ...",ZUMBI DOS PALMARES,Coliformes totais ...,...,AUSENTE ...,...
16274,1497,SAA,S130260000007,SISTEMA DE ABASTECIMENTO MANAUS ...,Serviço Municipal e outros ...,,MANAUS AMBIENTAL SA ...,...,2020,08,2020/08/18 00:00:00.000,2020/08/19 00:00:00.000,2020/09/14 11:05:59.000,SISTEMA DE DISTRIBUIÇÃO ...,Torneira antes da reservação ...,"RUA DR VALDIR CENTAURO, N 09 ...",ZUMBI DOS PALMARES,Escherichia coli ...,...,AUSENTE ...,...
16275,1226,SAA,S130260000007,SISTEMA DE ABASTECIMENTO MANAUS ...,Serviço Municipal e outros ...,,MANAUS AMBIENTAL SA ...,...,2020,07,2020/07/20 00:00:00.000,2020/07/21 00:00:00.000,2020/08/16 12:18:55.000,SISTEMA DE DISTRIBUIÇÃO ...,Torneira antes da reservação ...,"RUA ANTENOR CAVALCANTE, N 59 ...",ZUMBI DOS PALMARES,Coliformes totais ...,...,AUSENTE ...,...
16276,0582,SAA,S130260000007,SISTEMA DE ABASTECIMENTO MANAUS ...,Serviço Municipal e outros ...,,MANAUS AMBIENTAL SA ...,...,2020,05,2020/05/14 00:00:00.000,2020/05/15 00:00:00.000,2020/06/23 19:36:47.000,SISTEMA DE DISTRIBUIÇÃO ...,Torneira antes da reservação ...,"RUA JOÃO BOSCO BOUNIER, N 37 ...",ZUMBI DOS PALMARES,Coliformes totais ...,...,AUSENTE ...,...
